# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.enable('data_server')
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [ ]:
start_date = pd.to_datetime("2020-04-13")

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

In [ ]:
# Borrame
confirmados

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos a la fecha por cada grupo etáreo.

In [ ]:
alt.Chart(fallecidos_etareo).mark_bar().encode(
    y=alt.Y('fallecidos', axis=alt.Axis(title='Cantidad de Fallecidos')),
    x=alt.X('grupo_de_edad', axis=alt.Axis(title='Grupos Etarios'))
).properties(
    title='Total de fallecidos a la Fecha',
    width=300,
    height=400,
)

**Comentarios:** Podemos notar que la tercera edad tiene una mayor cantidad de fallecidos a causas del covid, especificamente los que se encuentran entre 70-79 años. 

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [ ]:
alt.Chart(activos).mark_bar().encode(
    y=alt.Y('poblacion', axis=alt.Axis(title='Poblacion')),
    x=alt.X('comuna', axis=alt.Axis(title='Comunas'))
).properties(
    title='Poblacion comunas de Chile',
    width=6500,
    height=400,
)

**Comentarios:** Podemos ver que es bastante variable, generalmente donde hay mayor población es en comunas conocidas , como Santiago, Antofagasta, etc...

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [ ]:
alt.Chart(fallecidos_etareo).mark_area().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Total de Fallecidos')),
    color='grupo_de_edad'
).properties(
    title='Evolucion fallecidos por grupo de Edad',
    width=970,
    height=400
)

**Comentarios:** Podemos notar que luego de mayo, aproximadamente, la cantidad de muertos crecio de forma dramatica, especialmente para los mayores de 50.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [ ]:
evo_confirmados = alt.Chart(confirmados).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('sum(casos_confirmados)', axis=alt.Axis(title='Total de casos confirmados')),
).properties(
    title='Evolucion casos confirmados',
    width=970,
    height=200
)

evo_fallecimientos = alt.Chart(fallecidos_etareo).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('sum(fallecidos)', axis=alt.Axis(title='Total de Fallecidos')),
).properties(
    title='Evolucion fallecidos',
    width=970,
    height=200
)

evo_activos = alt.Chart(activos).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('sum(casos_activos)', axis=alt.Axis(title='Total casos activos')),
).properties(
    title='Evolucion casos activos',
    width=970,
    height=200
)

display(evo_confirmados,
        evo_fallecimientos,  
        evo_activos)

**Comentarios:** De aqui podemos rescatar varias cosas: el total de casos confirmados y fallecido aumenta de manera pronunciada despues de mayo, y los casos activos tuvieron su pic en junio para luego disminuir de forma precipitada y mantenerse aproximadamente constante.

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [ ]:
alt.Chart(confirmados).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('average(tasa)', axis=alt.Axis(title='Tasa de incidencia')),
    color=alt.Color('region',
        scale=alt.Scale(scheme='category20'),
        legend=alt.Legend(direction='vertical')
    )
).properties(
    title='Evolucion tasa de incidencia por regiones',
    width=890,
    height=400
)

**Comentarios:** La mayor tasa de incidencia que podemos apreciar es en la region de Tarapacá, luego de aproximadamente la mitad de Julio, eso significa que se infecta una mayor cantidad de personas con respecto a su poblacion total. 

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [ ]:
data_13_abr = confirmados.query('fecha == "2020-04-13"') 
data_6_nov = confirmados.query('fecha == "2020-11-06"')

abril = alt.Chart(data_13_abr).mark_circle(size=60).encode(
    x=alt.X('casos_confirmados', axis=alt.Axis(title='Casos confirmados')),
    y=alt.Y('tasa', axis=alt.Axis(title='Tasa de incidencia')),
    color=alt.Color('region',
        scale=alt.Scale(scheme='category20b'),
        legend=alt.Legend(direction='vertical')
    )
).properties(
    title='13 de abril',
    width=380,
    height=400
)

nov = alt.Chart(data_6_nov).mark_circle(size=60).encode(
    x=alt.X('casos_confirmados', axis=alt.Axis(title='Casos confirmados')),
    y=alt.Y('tasa', axis=alt.Axis(title='Tasa de incidencia')),
    color=alt.Color('region',
        scale=alt.Scale(scheme='category20b'),
        legend=alt.Legend(direction='vertical')
    )
).properties(
    title='6 de noviembre',
    width=380,
    height=400
)

abril | nov

**Comentarios:** No consiero que sea necesario que el tamaño sea proporcional a la población ya que la tasa de incidencia ya resulta ser proporcional al tamaño de la poblacion. De los graficos podemos concluir que mientras aumenten los casos confirmados mayor resulta ser la tasa de incidencia, y tiene sentido ya que la tasa de incidencia corresponde a los casos confirmados proporcional a la poblacion cada 100000 habitantes.

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [ ]:
alt.Chart(activos).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos')),
    color=alt.Color('comuna',
        scale=alt.Scale(scheme='category20'),
        legend=alt.Legend(direction='vertical')
    )
).properties(
    title='Evolucion casos activos por Comuna',
    width=880,
    height=400
)

In [ ]:
region_names = [
    "region == 'Arica y Parinacota'", "region =='Tarapaca'",
    "region ==  'Antofagasta'", "region == 'Atacama'",
    "region == 'Coquimbo'", "region == 'Valparaiso'",
    "region == 'Metropolitana'", "region == 'Del Libertador General Bernardo O’Higgins'",
    "region == 'Maule'", "region == 'Nuble'", 
    "region ==  'Biobio'", "region == 'La Araucania'",
    "region ==  'La Araucania'", "region == 'Los Rios'",
    "region ==  'Los Lagos'", "region ==  'Aysen'",
    "region ==  'Magallanes y la Antartica'"
]

region = ['Arica y Parinacota', 'Tarapaca',
    'Antofagasta', 'Atacama',
    'Coquimbo', 'Valparaiso',
    'Metropolitana', 'Del Libertador General Bernardo O’Higgins',
    'Maule', 'Nuble', 
    'Biobio', 'La Araucania',
    'La Araucania', 'Los Rios',
    'Los Lagos', 'Aysen',
    'Magallanes y la Antartica'
]

for k in range(0,len(region_names)):     
    display(alt.Chart(activos.query(region_names[k])).mark_line().encode(
        x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
        y=alt.Y('casos_activos', axis=alt.Axis(title='Casos activos')),
        color=alt.Color('comuna',
        scale=alt.Scale(scheme='category20'),
        legend=alt.Legend(direction='vertical')
    )
    ).properties(
        title=region[k],
        width=880,
        height=400
    ))

**Comentarios:** Cuando hacemos la evolucion de las comunas en un solo grafico es dificil diferenciar todas las lineas ya que son muchas comunas, no alcanzan los colores y no se pueden apreciar los valores de forma satisfactoria, en cambio, si hacemos graficos por region, las que poseen pocas comunas salen beneficiadas, ya que los datos se pueden ver de mejor manera y alcanzan los colores, es decir que podemos obtener mejores conclusiones de los datos.
Aunque pase todo lo anterior, tener una gran cantidad de graficos puede ser pesado para la vista y para la memoria del computador, en cambio si se tiene uno es mas sencillo tener todo en un mismo lugar.

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [ ]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
}

region_names_activos = {
    "Tarapaca":"Tarapacá", 
    "Valparaiso":"Valparaíso", 
    "Metropolitana":"Metropolitana de Santiago",
    "Del Libertador General Bernardo O’Higgins":"Libertador General Bernardo O'Higgins", 
    "Nuble":"Ñuble",
    "Biobio":"Biobío", 
    "La Araucania":"La Araucanía", 
    "Los Rios":"Los Ríos", 
    "Aysen":"Aysén del General Carlos Ibáñez del Campo",
    "Magallanes y la Antartica":"Magallanes y de la Antártica Chilena"
}

In [ ]:
data_aux = (
            (confirmados[["casos_confirmados", "region"]].groupby(["region"]).agg(maximo_casos_confirmados = ("casos_confirmados", "max")).reset_index())
            .merge(
                activos[["casos_activos", "region"]].replace(region_names_activos).groupby(["region"])
                .agg(promedio_casos_activos = ("casos_activos", "mean")).reset_index(),
                how="inner",
                on="region")
            .merge(
                fallecidos[["fallecidos", "region"]].replace(region_names).groupby(["region"])
                .agg(maximo_fallecidos = ("fallecidos", "max")).reset_index(),
                how="inner",
                on="region")
)

com_fallecidos = alt.Chart(data_aux).mark_point(shape="cross", size=200).encode(
    x=alt.X('maximo_fallecidos', axis=alt.Axis(title='Maximo de casos confirmados(punto) y fallecidos(cruz)')),
    y=alt.Y('promedio_casos_activos', axis=alt.Axis(title='Promedio casos activos')),
    color=alt.Color('region',
        scale=alt.Scale(scheme='category20'),
        legend=alt.Legend(direction='vertical')),
).properties(
    title="Comparacion entre promedio de casos activos , máximo de casos confirmados y fallecidos",
    width=880,
    height=400
)

com_confirmados = alt.Chart(data_aux).mark_circle(size=160).encode(
    x=alt.X('maximo_casos_confirmados', axis=alt.Axis(title='Maximo de casos confirmados(punto) y fallecidos(cruz)')),
    y=alt.Y('promedio_casos_activos', axis=alt.Axis(title='Promedio casos activos')),
    color=alt.Color('region',
        scale=alt.Scale(scheme='category20'),
        legend=alt.Legend(direction='vertical'))
)

com_fallecidos + com_confirmados

**Comentarios:** Podemos darnos cuenta que generalmente a mayor promedio de casos activos la cantidad de confirmados es mayor, pero no ocurre lo mismo con el numero de fallecidos, lo que se debe a que en ciudades con menor numero de fallecidos hay un mejor control de la situacion.

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [ ]:
import geopandas as gpd
from pathlib import Path

In [ ]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

In [ ]:
type(regiones)

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [ ]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olvidar utilizar el diccionario `region_names`).

In [ ]:
casos_geo = (
    confirmados[["region", "region_id"]].drop_duplicates()
    .rename(columns={"region_id":"codigo_region"})
    .merge(
        (
            confirmados[["region",  "fecha", "poblacion", "casos_confirmados", "tasa"]]
            .groupby(['region', 'fecha'])
            .agg(poblacion = ('poblacion', 'sum'), casos_confirmados = ('casos_confirmados', 'sum'))
            .reset_index()
            .assign(tasa = lambda df: df['casos_confirmados'] / df['poblacion'] * 100000)
        ),
        how ="inner",
        on=["region"]
    )
    .merge(
        activos.groupby(["codigo_region", "fecha"]).agg(casos_activos = ("casos_activos", "sum")).reset_index(),
        how ="inner",
        on=["codigo_region", "fecha"]
    )
    .merge(
        fallecidos.replace(region_names),
        how="inner",
        on=["region", "fecha"]
    )
    .merge(
        regiones[["codregion", "geometry"]].rename(columns={"codregion": "codigo_region"}), 
        how="inner",
        on=["codigo_region"]
    )
)

casos_geo.head()

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
def covid_chile_chart(fecha, col):
    fecha = pd.to_datetime(fecha)
    data = casos_geo.loc[lambda df: df['fecha'] == fecha]
    
    chart = alt.Chart(data).mark_geoshape().encode(
        color=col
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [ ]:
col_widget = widgets.Dropdown(options=['poblacion', 
                                       'casos_confirmados',
                                       'tasa',
                                       'casos_activos',
                                       'fallecidos'], 
                              description='Columna', 
                              continuous_update=False)


In [ ]:
fecha_widget = widgets.DatePicker(description='Fecha',
                                  continuous_update=False)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** Vamos a comparar las fechas 2020-04-13 y 2020-11-13. Verificando como ha cambiado la tasa del 2020-04-13 al 2020-11-13 podemos ver que en la parte central de nuestro pais a disminuido notablemente, no asi en los extremos donde las tasas son elevadas. La poblacion con respectos a las fechas anteriores no ha tenido cambios bruscos. Los casos confirmados concentran su mayor cantidad en la region metropolitana en las dos fechas, pero con un aumento de casi 10000 veces la cantidas de abril. Para los casos activos, en abril podiamos ver se concentraba la mayor parte en la region metropolitana, en noviembre podemos ver un mayor numero de casos en el centro de nuestro pais. Y finalmente para los fallecidos, en noviembre el mayor numero, que era es casi 10000, se encuentra en la region metropolitana, pero antes, se repartia entre la zona central de chile. 